### Library and modules

In [1]:
import numpy as np, pandas as pd, geopandas as gpd, sklearn.metrics as metrics
import geemap, ee , rasterio, sankee, warnings, pickle
from rasterio.plot import show
from geopandas import GeoDataFrame
from osgeo import gdal, osr, gdalconst
import os, glob, subprocess
import rasterio
import matplotlib as mpl
import matplotlib.pyplot as plt, plotly.express as px
from geemap import ml
from sklearn import ensemble
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from shapely.geometry import Polygon, Point
from sklearn.metrics import confusion_matrix, cohen_kappa_score
warnings.filterwarnings('ignore')

### Earth Engine log in

In [ ]:
ee.Authenticate()

In [2]:
ee.Initialize()

In [3]:
######################## subset L8 collection   ########################################
##################################################################################
def getFactorImg(factorNames, image):
    factorList = image.toDictionary().select(factorNames).values()
    return ee.Image.constant(factorList)

######################## pandas normalise   ########################################
##################################################################################
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

######################## L8 cloud mask   ########################################
##################################################################################
def getQABits(image, start, end, mask):
    pattern = 0
    for i in range(start,end-1):
        pattern += 2**i
    return image.select([0], [mask]).bitwiseAnd(pattern).rightShift(start)


def maskQuality(image):
    QA = image.select('QA_PIXEL')
    shadowMask = getQABits(QA,3,3,'cloud_shadow')
    cloudMask = getQABits(QA,5,5,'cloud')
    cirrusMask = getQABits(QA,9,9,'cirrus_detected')
    scaleImg = getFactorImg(['REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'], image)
    offsetImg = getFactorImg(['REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'], image)
    scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg)
    
    return (image.addBands(scaled, None, True)
            .updateMask(cirrusMask)
            # .updateMask(cloudMask)
            # .updateMask(shadowMask)
            )



def prepSrL8(image):
    dilateMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 0)).eq(0)
    cirrusMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    cloudMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 3)).eq(0)
    shadowMask =  image.select('QA_PIXEL').bitwiseAnd(int('11111', 4)).eq(0)
    snowMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 5)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    scaleImg = getFactorImg(['REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'], image)
    offsetImg = getFactorImg(['REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'], image)
    scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg)

    return (image.addBands(scaled, None, True)
                 .updateMask(cirrusMask)
                 # .updateMask(dilateMask)
                 # .updateMask(cloudMask)
                 # .updateMask(shadowMask)
                 # .updateMask(snowMask)
                 #.updateMask(saturationMask)
           )

######################## L8 scale factors   ####################################
##################################################################################
def apply_scale_factors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

######################## convert L8 to SST   ####################################
##################################################################################
def sst(img):
    thermalBands = img.select('ST_TRAD')
    ln = k2_im.divide((k1_im.divide(thermalBands).add(1).log())).multiply(0.001).clip(site).rename("SST")
    #.convolve(kern)
    return img.addBands(ln)

######################## scale image   ####################################
##################################################################################
def scale(image):
    return ee.Image(image).multiply(0.001)

def addNDAVI(image):
    ndvi = image.normalizedDifference(['SR_B1', 'SR_B5']).rename('NDAVI')
    return image.addBands(ndvi)

def addNDWI(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDWI')
    return image.addBands(ndvi)

In [81]:
hab = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\BOSS\\Cleaned\\BOSS_clip_cluster.shp'))

sites = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\ICoAST sites\\WGS84\\ICOAST_sites.shp'))

kal = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\ICoAST sites\\Sites\\Kalbarri.shp'))

mask = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\Lidar mask\\Lidar_mask.shp'))

lidar_url = 'gs://cog-bucket/lidar_cog.tif'
lidar = geemap.load_GeoTIFF(lidar_url).rename('lidar')
tr = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\ICoAST sites\\Sites\\Two_rocks.shp'))
jur =  geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\ICoAST sites\\Sites\\Jurien.shp'))

#hab = hab.clip(mask)

### L8 collection

In [ ]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.centerObject(sites.geometry(), 5)
Map.addLayer(mask,  {}, 'Sites')
Map

In [140]:
start_date = '2021-01-01'
end_date = '2021-06-30'

L8 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(sites)
    .filterMetadata('CLOUD_COVER', 'less_than', 5)
    .filterDate(start_date, end_date)
    #.filter("IMAGE_QUALITY > 8")
    .map(addNDAVI)
    .map(addNDWI)
    .map(prepSrL8)
    .mean()
    .clip(sites)
    .addBands(lidar.rename('lidar'))
)

In [131]:
Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.centerObject(sites.geometry(), 5)
#Map.add_basemap('HYBRID')
Map.addLayer(L8, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'max': 0.1, 'min' : 0}, '2021')
Map.addLayer(mask,  {}, 'Sites')
Map.addLayer(hab,  {}, 'boss')
Map

Map(center=[-26.38096118269607, 113.62250401413422], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
glint = ee.FeatureCollection(Map.draw_features)
# glint = geemap.shp_to_ee(os.path.join(os.getcwd(),
#                         'Data\\Sunglint\\SG_poly.shp'))
#Map_glint.addLayer(glint, {}, 'Glint')
#Map.remove_drawn_features()
#geemap.ee_to_shp(glint,os.path.join(os.getcwd(),
#                      'Data\\Sunglint\\SG_poly.shp'))

In [21]:
B1 = L8.select(['SR_B5', 'SR_B1'])
B2 = L8.select(['SR_B5', 'SR_B2'])
B3 = L8.select(['SR_B5', 'SR_B3'])
B4 = L8.select(['SR_B5', 'SR_B4'])
#B8 = S2S.select(['B8', 'B8'])


lfitB1 = B1.reduceRegion(
              reducer=ee.Reducer.linearFit(),
              geometry=glint,
              scale=30,
              #maxPixels =  1000,
              bestEffort = True)

lfitB2 = B2.reduceRegion(
              reducer=ee.Reducer.linearFit(),
              geometry=glint,
              scale=30,
              #maxPixels =  1000,
              bestEffort = True)

lfitB3 = B3.reduceRegion(
              reducer=ee.Reducer.linearFit(),
              geometry=glint,
              scale=30,
              #maxPixels =  1000,
              bestEffort = True)

lfitB4 = B4.reduceRegion(
              reducer=ee.Reducer.linearFit(),
              geometry=glint,
              scale=30,
              #maxPixels =  1000,
              bestEffort = True)


slope_B1 = ee.Image.constant(lfitB1.get('scale')).clip(mask).rename('slope_B1')
slope_B2 = ee.Image.constant(lfitB2.get('scale')).clip(mask).rename('slope_B2')
slope_B3 = ee.Image.constant(lfitB3.get('scale')).clip(mask).rename('slope_B3')
slope_B4 = ee.Image.constant(lfitB4.get('scale')).clip(mask).rename('slope_B4')
#slope_B8 = ee.Image.constant(lfitB8.get('scale')).clip(coastal_waters).rename('slope_B8')
min_B5 = ee.Image.constant(L8.select('SR_B5').reduceRegion(ee.Reducer.min(),geometry = glint, scale = 30).get('SR_B5')).rename('min_B5')

glint_factors = ee.Image([slope_B1, slope_B2, slope_B3, slope_B4, min_B5])
L8_add = L8.addBands(glint_factors)


deglint_B1 = L8_add.expression(
    'Blue - (Slope * (NIR - MinNIR))', {
    'Blue': L8_add.select('SR_B1'),
    'NIR': L8_add.select('SR_B5'),
    'MinNIR': L8_add.select('min_B5'),
    'Slope': L8_add.select('slope_B1')
}).rename('SR_B1')

deglint_B2 = L8_add.expression(
    'Blue - (Slope * (NIR - MinNIR))', {
    'Blue': L8_add.select('SR_B2'),
    'NIR': L8_add.select('SR_B5'),
    'MinNIR': L8_add.select('min_B5'),
    'Slope': L8_add.select('slope_B2')
}).rename('SR_B2')

deglint_B3 = L8_add.expression(
    'Green - (Slope * (NIR - MinNIR))', {
    'Green': L8_add.select('SR_B3'),
    'NIR': L8_add.select('SR_B5'),
    'MinNIR': L8_add.select('min_B5'),
    'Slope': L8_add.select('slope_B3')
}).rename('SR_B3')

deglint_B4 = L8_add.expression(
    'Red - (Slope * (NIR - MinNIR))', {
    'Red': L8_add.select('SR_B4'),
    'NIR': L8_add.select('SR_B5'),
    'MinNIR': L8_add.select('min_B5'),
    'Slope': L8_add.select('slope_B4')
}).rename('SR_B4')



L8_deglint = ee.Image([deglint_B1, deglint_B2, deglint_B3, deglint_B4])
L8_deglint = L8_deglint.addBands(L8.select(['SR_B5']))
L8_deglint = L8_deglint.addBands(L8.select(['SR_B6']))
L8_deglint = addNDAVI(L8_deglint)
L8_deglint = addNDWI(L8_deglint)

In [22]:
Map.addLayer(L8_deglint, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'max': 0.1, 'min' : 0}, 'Deglint')
Map

Map(bottom=621436.0, center=[-31.527043924837933, 115.74062347412111], controls=(WidgetControl(options=['posit…

In [141]:
start_date = '2013-01-01'
end_date = '2013-06-30'

L8_13 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(sites)
    .filterMetadata('CLOUD_COVER', 'less_than', 5)
    .filterDate(start_date, end_date)
    #.filter("IMAGE_QUALITY > 7")
    .map(addNDAVI)
    .map(addNDWI)
    .map(prepSrL8)
    .mean()
    .clip(sites)
    #.addBands(lidar.rename('lidar'))
)

In [132]:
# Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
# Map.centerObject(sites.geometry(), 5)
Map.addLayer(L8_13, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'max': 0.1, 'min' : 0}, '2013')
Map

Map(center=[-26.38096118269607, 113.62250401413422], controls=(WidgetControl(options=['position', 'transparent…

In [142]:
start_date = '2011-01-01'
end_date = '2011-12-30'

L5 = (
    ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
    .filterBounds(sites)
    .filterMetadata('CLOUD_COVER', 'less_than', 5)
    #.filter("IMAGE_QUALITY == 9")
    .filterDate(start_date, end_date)
    .map(prepSrL8)
    .mean()
    .clip(sites)
)

In [143]:
# Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
# Map.centerObject(sites.geometry(), 5)
# Map.add_basemap('HYBRID')
Map.addLayer(L5, {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'max': 0.1, 'min' : 0}, '2010')
Map.addLayer(sites,  {}, 'Sites')
Map

Map(bottom=1233957.0, center=[-30.258029283193746, 115.05260467529298], controls=(WidgetControl(options=['posi…

In [31]:
proj = ee.Projection('EPSG:4326')
ndavi_dat = geemap.extract_values_to_points(hab,
                                            L8.select('NDAVI'), 
                                            crs = proj, 
                                            scale = 30)

ndwi_dat = geemap.extract_values_to_points(hab,
                                            L8.select('NDWI'), 
                                            crs = proj, 
                                            scale = 30)

b1_dat = geemap.extract_values_to_points(hab,
                                         L8.select('SR_B1'), 
                                         crs = proj, 
                                         scale = 30)

gr_dat = geemap.extract_values_to_points(hab,
                                         L8.select('SR_B2'), 
                                         crs = proj, 
                                         scale = 30)

lid_dat = geemap.extract_values_to_points(hab,
                                         L8.select('lidar'), 
                                         crs = proj, 
                                         scale = 30)

ndavi_dat  = geemap.ee_to_pandas(ndavi_dat)
b1_dat  = geemap.ee_to_pandas(b1_dat)
ndwi_dat = geemap.ee_to_pandas(ndwi_dat)
gr_dat = geemap.ee_to_pandas(gr_dat)
lid_dat = geemap.ee_to_pandas(lid_dat)

In [32]:
#ndwi_dat.info()
ndavi_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   first       191 non-null    float64
 1   date        192 non-null    object 
 2   broad_Ston  192 non-null    int64  
 3   image       192 non-null    object 
 4   Total_adj   192 non-null    int64  
 5   broad_Unkn  192 non-null    int64  
 6   visibility  192 non-null    object 
 7   broad_Spon  192 non-null    int64  
 8   latitude    192 non-null    float64
 9   fov_Limite  192 non-null    float64
 10  Cluster     192 non-null    object 
 11  broad_Seag  192 non-null    int64  
 12  mean_relie  192 non-null    float64
 13  sample      192 non-null    object 
 14  sd_relief   192 non-null    float64
 15  site        192 non-null    object 
 16  time_botto  192 non-null    object 
 17  broad_Macr  192 non-null    int64  
 18  location    192 non-null    object 
 19  broad_Cons  192 non-null    i

In [50]:
ndavi_dat = ndavi_dat.rename(columns={'first': 'NDAVI'})
b1_dat = b1_dat.rename(columns={'first': 'SR_B1'})
ndwi_dat = ndwi_dat.rename(columns={'first': 'NDWI'})
gr_dat = gr_dat.rename(columns={'first': 'SR_B2'})
lid_dat = lid_dat.rename(columns={'first': 'lidar'})

train_df = ndavi_dat
train_df['SR_B1'] = b1_dat['SR_B1']
train_df['NDWI'] = ndwi_dat['NDWI']
train_df['SR_B2'] = gr_dat['SR_B2']
train_df['lidar'] = lid_dat['lidar']
train_split = train_df.sample(frac=0.8,random_state=2)
train_split = train_split.dropna(subset=['NDAVI', 'SR_B1', 'NDWI', 'Cluster', 'lidar'])
test_split = train_df.drop(train_split.index)
test_split = test_split.dropna(subset=['NDAVI', 'SR_B1', 'NDWI', 'Cluster', 'lidar'])

feature_names = ['NDAVI', 'SR_B1', 'NDWI', 'SR_B2', 'lidar']
label = "Cluster"
X = train_split[feature_names]
y = train_split[label]

In [55]:
n_trees = 1000
rf = (
     ensemble.RandomForestClassifier(n_trees,
                                    max_features= 'sqrt',
                                    max_depth = 7,
                                    ccp_alpha=0.001,
                                    oob_score = True)
                                    .fit(X, y)
)

pred = rf.predict(test_split[feature_names])

In [56]:
confusion_matrix(test_split[label], pred)
cohen_kappa_score(test_split[label], pred)

0.2964205816554809

In [57]:
trees = ml.rf_to_strings(rf, 
                        feature_names,
                        processes=3,
                        output_mode = 'CLASSIFICATION')
ee_classifier = ml.strings_to_classifier(trees)
classified = L8.select(feature_names).classify(ee_classifier)

In [58]:
ee_classifier = ml.strings_to_classifier(trees)
classified = L8.select(feature_names).classify(ee_classifier)

In [59]:
Map_pred = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map_pred.add_basemap('HYBRID')
vis_pred = {
    'min': 0,
    'max': 1,
    'palette': ['#9EE953', '#D2B866', '#3A570B'],
}

Map_pred.addLayer(classified, vis_pred  ,'classification',
)

Map_pred


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
Kal_L8 = L8.select(['SR_B2', 'SR_B3', 'SR_B4']).clip(kal).unmask()
task_Kal_L8 = ee.batch.Export.image.toCloudStorage(**{
    'image': Kal_L8 ,
    'description': 'L8_Kal',
    'fileNamePrefix' :'L8_Kal',
    'crs': 'EPSG:4326',
    'scale': 30,
    'region': kal.geometry(),
    'fileFormat': 'GeoTIFF',
    'bucket': 'cog-bucket',
    'maxPixels': 1e13,
    'skipEmptyTiles' : True,
    'formatOptions': {'cloudOptimized': True}
})
task_Kal_L8.start()
task_Kal_L8.status()

In [ ]:
task_Kal_L8.status()

In [ ]:
Kal_L8_13 = L8_13.select(['SR_B2', 'SR_B3', 'SR_B4']).clip(kal).unmask()
task_Kal_L8_13 = ee.batch.Export.image.toCloudStorage(**{
    'image': Kal_L8_13 ,
    'description': 'L8_13_Kal',
    'fileNamePrefix' :'L8_13_Kal',
    'crs': 'EPSG:4326',
    'scale': 30,
    'region': kal.geometry(),
    'fileFormat': 'GeoTIFF',
    'bucket': 'cog-bucket',
    'maxPixels': 1e13,
    'skipEmptyTiles' : True,
    'formatOptions': {'cloudOptimized': True}
})
task_Kal_L8_13.start()
task_Kal_L8_13.status()

In [ ]:
task_Kal_L8_13.status()

In [ ]:
Kal_L5 = L5.select(['SR_B1', 'SR_B2', 'SR_B3']).clip(kal).unmask()
task_Kal_L5 = ee.batch.Export.image.toCloudStorage(**{
    'image': Kal_L5 ,
    'description': 'L5_Kal',
    'fileNamePrefix' :'L5_Kal',
    'crs': 'EPSG:4326',
    'scale': 30,
    'region': kal.geometry(),
    'fileFormat': 'GeoTIFF',
    'bucket': 'cog-bucket',
    'maxPixels': 1e13,
    'skipEmptyTiles' : True,
    'formatOptions': {'cloudOptimized': True}
})
task_Kal_L5.start()
task_Kal_L5.status()

In [ ]:
task_Kal_L5.status()

In [70]:
g = ee.Geometry(mask.geometry())

In [90]:
training = L8.select(['SR_B2','SR_B3','SR_B4']).clip(jur).sample(
    **{
        'region': jur.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

# Instantiate the clusterer and train it.
n_clusters = 2
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)
# Cluster the input using the trained clusterer.
result = L8.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(bottom=1233982.0, center=[-30.261736090037466, 115.04934310913087], controls=(WidgetControl(options=['posi…